In [ ]:
# Импорт необходимых модулей
from tracking.face_reid import FaceReId
from utils.scene_tools import (
   enrich_scenes_with_characters,
   enrich_scenes_with_audio,
   clean_and_merge_short_scenes,
   print_scenes_formatted, cluster_scenes_with_time_windows, resolve_time_overlaps,
)
from sentence_transformers import SentenceTransformer

from utils.video_tools import save_scenes_ffmpeg
from analyzers.video.video_analyzer import VideoAnalyzer
from analyzers.audio_analyzer import AudioSceneAnalyzer
import os
#from video.render_video import render_video_with_faces
from analyzers.video_analyzer_complex import VideoPipeline
import pickle

c:\Users\d.krasnov.kulturauao\Projects\My_projects\Hachakaton\kion_practice\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
VIDEO_PATH = "videos/Video_01.mp4"          # Путь к видеофайлу
AUDIO_PATH = "temp/temp_audio.wav"           # Временный WAV
OUTPUT_DIR = "output_files/"
TEMP_DIR = "temp/"         # Папка для результатов
CACHE_PATH = os.path.join(TEMP_DIR, "results.pkl")
MIN_SCENE_LENGTH = 2.0                  # Мин. длительность сцены
MAX_SCENE_LENGTH = 300.0                # Макс. допустимая длина сцены (5 мин)


# Анализ видео и выделение сцен

### В этой ячейке мы создаём объект VideoAnalyzer и анализируем видео для получения сцены и лиц, привязанных к трекам.


In [ ]:
if os.path.exists(CACHE_PATH):
    print("🔁 Загружаем из кеша...")
    with open(CACHE_PATH, "rb") as f:
        scenes, track_faces, tracking_frames, track_id_to_person = pickle.load(f)

else:
    pipeline = VideoPipeline()
    scenes, track_faces, tracking_frames, track_id_to_person = pipeline.run(VIDEO_PATH, OUTPUT_DIR)

    with open(CACHE_PATH, "wb") as f:
        pickle.dump((scenes, track_faces, tracking_frames, track_id_to_person), f)

# Обогащение сцен информацией о персонажах

### Связываем сцены с именами персонажей на основе сопоставления track_id → персонаж.


In [ ]:
scene_data = enrich_scenes_with_characters(scenes, track_id_to_person)


# Анализ аудио: распознавание речи и активность

### Используем AudioAnalyzer для транскрибирования речи и измерения активности звука (энергии).

In [ ]:
splitter = AudioSceneAnalyzer(
    video_path=VIDEO_PATH,
    output_dir=OUTPUT_DIR,
    whisper_model_size='small'
)
splitter.run(sensitivity=0.88, min_scene_duration=2.0)
segments = splitter.transcribe_audio()
energy = splitter.detect_audio_activity(frame_duration=1.0)

# Обогащение сцен транскриптами и аудио энергией

### Добавляем к сценам распознанный текст и усреднённые показатели звуковой активности.


In [ ]:
scene_data_copy = scene_data.copy()

scenes_enrich = enrich_scenes_with_audio(scene_data_copy, segments, energy)


# Семантическая группировка и очистка сцен

### Группируем сцены по сходству персонажей, текстов и аудио, затем объединяем слишком короткие сцены.


In [ ]:
scenes_enrich_copy = scenes_enrich.copy()

# Загружаем модель для вычисления семантических эмбеддингов текста
sentenceTransformer = SentenceTransformer('all-mpnet-base-v2', device="cuda")

# Группируем похожие сцены
scenes_grouped = cluster_scenes_with_time_windows(scenes_enrich_copy, sentenceTransformer)

grouped_chapters_no_overlap = resolve_time_overlaps(scenes_grouped)

# Убираем слишком короткие сцены, объединяя их с соседними
scenes_cleaned = clean_and_merge_short_scenes(grouped_chapters_no_overlap, min_duration=2.0, min_words=3)

# Вывод результата

### Печатаем красиво отформатированный отчёт по сценам.


In [ ]:
print_scenes_formatted(scenes_cleaned)


# Сохранение выделенных сцен в отдельные видеофайлы

### Используем ffmpeg для вырезки и сохранения сцен в отдельные файлы.


In [ ]:
output_dir = "output_scenes"

save_scenes_ffmpeg(video_path, scenes_cleaned, output_dir)

In [ ]:
import os
import pickle
from process_video.video_pipeline import VideoPipeline  # путь обнови под себя

video_path = "test4.mp4"
output_path = "output_annotated4.mp4"
cache_path = "results2.pkl"

if os.path.exists(cache_path):
    print("🔁 Загружаем из кеша...")
    with open(cache_path, "rb") as f:
        track_faces, tracking_frames, track_id_to_person = pickle.load(f)

    pipeline = VideoPipeline(using_cache=True)
    pipeline.render_labeled_video(VIDEO_PATH, output_path, tracking_frames, track_id_to_person)

else:
    pipeline = VideoPipeline()
    track_faces, tracking_frames, track_id_to_person = pipeline.run(VIDEO_PATH, output_path)

    with open(cache_path, "wb") as f:
        pickle.dump((track_faces, tracking_frames, track_id_to_person), f)